<a href="http://landlab.github.io"><img style="float: left" src="../../landlab_header.png"></a>

# A Tutorial on Coupling the River Bed Dynamics and Overland Flow Components

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>

This notebook illustrates the process of running the river bed dynamics component coupled to the 2D deAlmeida overland flow component in an initially flat bedriver that receives an excess sediment supply. Then, it shows how the bed slope is adjusted continously in time and space until it reaches a new bed elevation equilibrium.

First, let's import the necessary libraries and modules:

In [ ]:
import copy

import numpy as np
from matplotlib import pyplot as plt

from landlab import imshow_grid
from landlab.components import OverlandFlow, RiverBedDynamics
from landlab.grid.mappers import map_mean_of_link_nodes_to_link
from landlab.io import read_esri_ascii

Next, let's define some numerical simulation parameters, time control settings, filenames for accessing the grain size distribution, the digital elevation model (DEM), and set boundary conditions.

In [ ]:
# ASCII raster DEM containing the bed surface elevation
zDEM = "bedElevationDEM.asc"

# ASCII file containing grain size distribution
gsd = np.loadtxt("bed_gsd.txt")

# Elapsed time (sec)
t = 0

# Maximum time step in sec
max_dt = 5

# Maximum simulation time (sec)
sim_max_t = 5 * 86400

n = 0.03874  # Manning's n

# bedload rate at inlet in m3/s
in_qb = -0.0087

# Link Id in which sediment supply and discharge enters
in_l = np.array((221, 222))

# Nodes Id in Water depth is specified
in_n = np.array((129, 130))

# Node ID for fixed Nodes
fixed_nodes_id = np.array((1, 2, 5, 6))

Now we create fields and instantiate the OverlandFlow component. Let's examine the fields that are mandatory for the OverlandFlow component.

In [ ]:
OverlandFlow.input_var_names  # Gives the list of all required fields

Therefore, we need to create the surface_water__depth and topographic__elevation fields. Additionally, let's create a copy of the topographic elevation to use for comparison at the end of the simulation.

In [ ]:
# Load the topographic elevation from a DEM and creates the topographic__elevation field
(rmg, z) = read_esri_ascii(zDEM, name="topographic__elevation")

# Initialize the surface water depth field - Creates the surface_water__depth
rmg.add_zeros("surface_water__depth", at="node")

# A copy of the original topographic__elevation - This will be used in a plot at the end so it is copied as a grid field
rmg["node"]["topographic__elevation_original"] = copy.deepcopy(
    rmg["node"]["topographic__elevation"]
)

It would be beneficial to visualize the geometry at this stage

In [ ]:
imshow_grid(rmg, "topographic__elevation", vmin=0, vmax=45)

Now, let's instantiate the OverlandFlow component.

In [ ]:
of = OverlandFlow(
    rmg,
    h_init=0.001,
    mannings_n=n,
)

Now it is time to instantiate the RiverBedDynamics component, to check the required fields we do:

In [ ]:
RiverBedDynamics.input_var_names

We need to create or make sure that these fields exist: 
'surface_water__depth'
'surface_water__velocity'
'topographic__elevation'

Notice that 'topographic__elevation' was already created. In this case, 'surface_water__depth' appears again, but we should verify if it defined in links or nodes. Also, 'surface_water__velocity' has to be created. We can use RiverBedDynamics.var_mapping to check where these variables are mapped.

In [ ]:
RiverBedDynamics.var_mapping

We have 'surface_water__depth' and 'topographic__elevation' at nodes. Therefore, we will map 'surface_water__depth' to links and create 'surface_water__velocity'

In [ ]:
rmg.add_zeros("surface_water__velocity", at="link")
rmg["link"]["surface_water__depth"] = map_mean_of_link_nodes_to_link(
    rmg, "surface_water__depth"
)

Before instantiating RiverBedDynamics, we need to specify for this case the sediment transport and bed elevation boundary conditions. These are optional fields, that is why they are not listed as mandatory. We will use the bed_surface__elevation_fixed_node and sediment_transport__sediment_supply_imposed_link optional fields. First we create these fields.

In [ ]:
# Creates optional fields that will be used in the simulation
# fixed_nodes defines as 1 if a node is fixed or 0 if it can varies in elevation
fixed_nodes = np.zeros_like(z)
fixed_nodes[fixed_nodes_id] = 1  # Assigns 1 to fixed nodes, all the others are zero.

# sediment_transport__sediment_supply_imposed
qb = np.full(rmg.number_of_links, 0.0)
qb[in_l] = in_qb  # Previously defined - # bedload rate at inlet in m3/s

We will pass fixed_nodes and qb during instantiation.

In [ ]:
rbd = RiverBedDynamics(
    rmg,
    gsd=gsd,
    variable_critical_shear_stress=True,
    outlet_boundary_condition="fixedValue",
    bed_surface__elevation_fixed_node=fixed_nodes,
    sed_transport__bedload_rate_imposed_link=qb,
)

We set boundaries as closed boundaries, the outlet is set to an open boundary

In [ ]:
rmg.set_watershed_boundary_condition_outlet_id([1, 2], z, 45.0)

A little explanation of what we did for the watershed boundary condition. Our DEM represents a long watershed with a given slope in the central region. The outlet is in the south boundary, which according to landlab grid convention is defined as nodes 1 and 2 (that is why we have ...outlet_id([1,2]..., then z contains the bed surface elevation created when we did:
(rmg, z) = read_esri_ascii(bedElevation, name='topographic__elevation'), and 45 tells the component that if that value exist in the DEM it is an invalid value. Most commonly it is used as -9999.

Here we define some ghost cells. In this tutorial we are analyzing a 1500 m long reach, but the DEM contains data up to 1650 m. Outside 1500 m, where is the point in which the sediment supply is defined, the bed elevation is assumed to vary according to a gradient preserving condition, a boundary condition that allows the water to flow without problems. We will called these nodes calc_n_Id

In [ ]:
calc_n_Id = np.arange(128, 136)
n_col = rmg.number_of_node_columns
n_row_calc_n = int(calc_n_Id.shape[0] / n_col)
calc_n_Id = np.reshape(calc_n_Id, (n_row_calc_n, n_col))

Now, let's execute the loop that drives the simulation for both components:

In [ ]:
# The simulation may take a long time to run,
# so we added a progress report
progress0 = 0
while t < sim_max_t:
    # Boundary conditions of discharge and flow depth

    # Flow discharge in m3/s/m
    rmg["link"]["surface_water__discharge"][in_l] = -1

    # Flow depth at nodes in m
    rmg["node"]["surface_water__depth"][in_n] = 0.45

    # Flow depth at links in m
    rmg["link"]["surface_water__depth"][in_l] = 0.45

    # Velocity at previous time
    rbd._surface_water__velocity_prev_time_link = (
        rmg["link"]["surface_water__discharge"] / rmg["link"]["surface_water__depth"]
    )

    of.overland_flow(dt=max_dt)  # Runs overland flow for one time step

    # Velocity at current time
    rmg["link"]["surface_water__velocity"] = (
        rmg["link"]["surface_water__discharge"] / rmg["link"]["surface_water__depth"]
    )

    rbd.run_one_step()  # Runs riverBedDynamics for one time step

    # Gradient preserving at upstream ghost cells
    dsNodesId = np.array(calc_n_Id[0, 1] - np.arange(1, 3) * n_col)

    # Updated topographic elevation
    z = rmg["node"]["topographic__elevation"]

    bedSlope = (z[dsNodesId[0]] - z[dsNodesId[1]]) / rmg.dx

    for i in np.arange(0, calc_n_Id.shape[0]):
        rmg["node"]["topographic__elevation"][calc_n_Id[i, 1 : n_col - 1]] = (
            z[calc_n_Id[i, 1 : n_col - 1] - 2 * n_col] + 2 * rmg.dx * bedSlope
        )

    t += of.dt
    progress = int((t / sim_max_t) * 100)
    if progress > progress0 + 1:
        print("\r" + f"Progress: [{progress}%]", end="")
        progress0 = progress

Let's take a look at the new topography and the water depth field

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))

plt.sca(axs[0])
imshow_grid(rmg, "topographic__elevation", vmin=0, vmax=45)
plt.title("Topographic elevation original[m]")

plt.sca(axs[1])
diff_z = (
    rmg["node"]["topographic__elevation"]
    - rmg["node"]["topographic__elevation_original"]
)
imshow_grid(rmg, diff_z, vmin=0, vmax=4.0)
plt.title("Difference in elevation [m]")  # set a title

plt.sca(axs[2])
imshow_grid(rmg, "surface_water__depth", cmap="Blues")
plt.title("Surface water depth [m]")

plt.tight_layout()
plt.show()

After five days, we can observe significant changes in the riverbed. Deposition is predominantly seen in the upper part. Stable riverbed conditions for this case are achieved in approximately 160 days. To observe the most significant changes under these experimental conditions, you should set the simulation_max_time to 160*86400 seconds.

### Click here for more <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>